#### Gộp Vietnam Airlines và Vietjet Air và Gộp 3 hãng thành 1 


In [2]:
import pandas as pd
import os

# Cài đặt hiển thị đẹp
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

# Đường dẫn thư mục chứa file
RAW_PATH = "D:\\E\\raw_data"

def load_and_prepare(files, drop_airline=False, drop_is_duplicate=False):
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(RAW_PATH, file))
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
        
        if drop_airline and "airline" in df.columns:
            df.drop(columns="airline", inplace=True)
        if drop_is_duplicate and "is_duplicate" in df.columns:
            df.drop(columns="is_duplicate", inplace=True)
        
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# 1. Vietjet: xoá airline & is_duplicate
df_vietjet = load_and_prepare(["vietjet_1.csv", "vietjet_2.csv"],
                              drop_airline=True, drop_is_duplicate=True)

# 2. Vietnam Airlines: xoá airline
df_vnairlines = load_and_prepare(["VNairlines_1_1.csv", "VNairlines_2_2.csv"],
                                 drop_airline=True)

# 3. Bamboo: giữ nguyên hoàn toàn
df_bamboo = load_and_prepare(["BambooAirway.csv"])

# Gộp tất cả
df_all = pd.concat([df_vietjet, df_vnairlines, df_bamboo], ignore_index=True)

# Xem kết quả
print("Tổng số dòng sau khi gộp:", len(df_all))
print(df_all.head(10))


Tổng số dòng sau khi gộp: 2255788
                     source.name        date    route                        flight_id origin destination fare_class fare_code     price_vnd          taxes departure_time arrival_time booking_class  \
0  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ568-DLIDAD-2025-1-01-1450    DLI         DAD        EL1   QPXVNWS  1.860040e+06  572426.357322            NaN          NaN             Z   
1  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ886-DLIDAD-2025-1-01-1450    DLI         DAD        EC1     ZVNWP  1.929579e+06  577492.077298            NaN          NaN             Z   
2  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ430-DLIDAD-2025-1-01-1450    DLI         DAD        EF1     ZVNWP  2.148786e+06  594070.797220            NaN          NaN             Z   
3  DLI-DAD_01-06-25_31-12-25.csv  2025-06-13  DLI_DAD  SEG-VJ180-DLIDAD-2025-1-01-1450    DLI         DAD        DC1     ZVNWP  2.222009e+06  599136.517197       

In [3]:
# Kiểm tra có bao nhiêu hãng bay 
unique_airlines = df_all['operating_airline_name'].dropna().unique()
print("Số lượng hãng bay duy nhất trong 'operating_airline_name':", len(unique_airlines))
print("🛫 Danh sách các hãng bay:")
print(unique_airlines)


Số lượng hãng bay duy nhất trong 'operating_airline_name': 3
🛫 Danh sách các hãng bay:
['Vietjet Air' 'VietNam Airlines' 'BAMBOO AIRWAYS']


### Lọc giữ lại các cột cần thiết 

In [4]:
df_all.drop(columns=['source.name', 'departure_time', 'arrival_time', 'status'], inplace=True)
print("Đã xóa các cột: source.name, departure_time, arrival_time, status khỏi df_all.")


Đã xóa các cột: source.name, departure_time, arrival_time, status khỏi df_all.


In [6]:

print("🧾 Các cột hiện có trong df_all:")
print(df_all.columns.tolist())


🧾 Các cột hiện có trong df_all:
['date', 'route', 'flight_id', 'origin', 'destination', 'fare_class', 'fare_code', 'price_vnd', 'taxes', 'booking_class', 'cabin', 'quota', 'service_code', 'fare_conditions', 'operating_airline_code', 'operating_airline_name']


In [7]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2255788 entries, 0 to 2255787
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   date                    object 
 1   route                   object 
 2   flight_id               object 
 3   origin                  object 
 4   destination             object 
 5   fare_class              object 
 6   fare_code               object 
 7   price_vnd               float64
 8   taxes                   float64
 9   booking_class           object 
 10  cabin                   object 
 11  quota                   float64
 12  service_code            object 
 13  fare_conditions         object 
 14  operating_airline_code  object 
 15  operating_airline_name  object 
dtypes: float64(3), object(13)
memory usage: 275.4+ MB


In [8]:
# Ép kiểu cột date về datetime
df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')

In [9]:
# Tạo cột giờ
df_all['hour'] = df_all['date'].dt.hour

# Tạo cột "khoảng thời gian bay"
def get_time_slot(hour):
    if 5 <= hour < 12:
        return 'Sáng'
    elif 12 <= hour < 17:
        return 'Chiều'
    elif 17 <= hour < 21:
        return 'Tối'
    else:
        return 'Đêm'

df_all['time_slot'] = df_all['hour'].apply(get_time_slot)

In [10]:
# Số chuyến bay tổng cộng
print("Tổng số chuyến bay:", len(df_all))

# Số chuyến theo hãng
print("\nSố chuyến theo hãng:")
print(df_all['operating_airline_name'].value_counts())

# Thống kê giá vé
print("\nThống kê giá vé (price_vnd):")
print(df_all['price_vnd'].describe())

Tổng số chuyến bay: 2255788

Số chuyến theo hãng:
operating_airline_name
VietNam Airlines    1164507
Vietjet Air         1067108
BAMBOO AIRWAYS        23790
Name: count, dtype: int64

Thống kê giá vé (price_vnd):
count    2.255405e+06
mean     4.644202e+06
std      3.051185e+06
min      4.107838e+05
25%      2.448000e+06
50%      3.849000e+06
75%      5.996006e+06
max      1.592100e+07
Name: price_vnd, dtype: float64


In [11]:
# Đảm bảo date đã đúng kiểu datetime
df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')

# Thêm các cột thời gian
df_all['month'] = df_all['date'].dt.month
df_all['weekday'] = df_all['date'].dt.day_name()
df_all['week'] = df_all['date'].dt.isocalendar().week

In [12]:
print("\nGiá vé trung bình theo tháng:")
print(df_all.groupby('month')['price_vnd'].mean().round(0))


Giá vé trung bình theo tháng:
month
1.0     3344166.0
2.0     3995363.0
3.0     3406687.0
4.0     3349370.0
5.0     3333594.0
6.0     2652879.0
7.0     2562875.0
8.0     2564037.0
9.0     2571280.0
10.0    2608576.0
11.0    2665188.0
12.0    2673173.0
Name: price_vnd, dtype: float64


In [13]:
print("\nGiá vé trung bình theo ngày trong tuần:")
print(df_all.groupby('weekday')['price_vnd'].mean().round(0))



Giá vé trung bình theo ngày trong tuần:
weekday
Friday       2833743.0
Monday       2790339.0
Saturday     2788286.0
Sunday       2881056.0
Thursday     2775568.0
Tuesday      2736647.0
Wednesday    2728504.0
Name: price_vnd, dtype: float64


In [14]:
print("\nGiá vé trung bình theo tuần:")
print(df_all.groupby('week')['price_vnd'].mean().round(0))



Giá vé trung bình theo tuần:
week
1     2943236.0
2     3299986.0
3     3376702.0
4     3371631.0
5     3617389.0
6     3925365.0
7     3920435.0
8     4287099.0
9     3997641.0
10    3448884.0
11    3369223.0
12    3372469.0
13    3368340.0
14    3347177.0
15    3346089.0
16    3346985.0
17    3351993.0
18    3355261.0
19    3348176.0
20    3348420.0
21    3332054.0
22    2920749.0
23    2633707.0
24    2690329.0
25    2671333.0
26    2610581.0
27    2574673.0
28    2576106.0
29    2568302.0
30    2555714.0
31    2557170.0
32    2584649.0
33    2524901.0
34    2515611.0
35    2624998.0
36    2663373.0
37    2546230.0
38    2556816.0
39    2543321.0
40    2588707.0
41    2589535.0
42    2602873.0
43    2614807.0
44    2654876.0
45    2658449.0
46    2660212.0
47    2668809.0
48    2656397.0
49    2671626.0
50    2661428.0
51    2673795.0
52    2672254.0
Name: price_vnd, dtype: float64
